In [30]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [51]:
address = 'Srinagar India' # where yo want to find
Query = 'Hospital'  # what you are looking for

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of your location are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of your location are 34.0747444, 74.8204443.


In [10]:
CLIENT_ID = 'ZMFOM3FAOXDSVNT1BFYGBMDAUE5332L1BLQ415PHNVL45L2I' # your Foursquare ID
CLIENT_SECRET = 'SKSEZ0NIMH4KXZWEOFWIAJOKWFZTPRSMCLJZFVWUCDJI3OET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZMFOM3FAOXDSVNT1BFYGBMDAUE5332L1BLQ415PHNVL45L2I
CLIENT_SECRET:SKSEZ0NIMH4KXZWEOFWIAJOKWFZTPRSMCLJZFVWUCDJI3OET


In [52]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    Query 
    )
url # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=ZMFOM3FAOXDSVNT1BFYGBMDAUE5332L1BLQ415PHNVL45L2I&client_secret=SKSEZ0NIMH4KXZWEOFWIAJOKWFZTPRSMCLJZFVWUCDJI3OET&v=20180605&ll=34.0747444,74.8204443&query=Hospital'

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8c9bab83525f001b34f3b5'},
 'response': {'venues': [{'id': '536a31fb498e3ba5d4dc582c',
    'name': 'Modern Hospital',
    'location': {'lat': 34.06950075150586,
     'lng': 74.82684720194297,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.06950075150586,
       'lng': 74.82684720194297}],
     'distance': 830,
     'cc': 'IN',
     'country': 'India',
     'formattedAddress': ['India']},
    'categories': [{'id': '4bf58dd8d48988d104941735',
      'name': 'Medical Center',
      'pluralName': 'Medical Centers',
      'shortName': 'Medical',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586273252',
    'hasPerk': False},
   {'id': '50277286e4b00bbc7a657b1c',
    'name': 'Govt. Chest Disease Hospital',
    'location': {'lat': 34.07670211791992,
     'lng': 74.83289337158203,
     'labeledLatLngs': [{'label': 'display',
       

In [54]:
venues = results['response']['venues']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'location.lat', 'location.lng','location.address','location.distance']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
#nearby_venues['venue.name'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(15)

,name,lat,lng,address,distance
0,Modern Hospital,34.069501,74.826847,NaN,830
1,Govt. Chest Disease Hospital,34.076702,74.832893,NaN,1168
2,ld hospital,34.065500,74.808699,NaN,1493
3,Apollo Hospital,34.080196,74.798950,NaN,2072
4,G.B.Pant Children Hospital,34.070618,74.844101,NaN,2229
5,Army Hospital,34.069263,74.843760,Badami Bagh,2234
6,government dental college and hospital,34.087300,74.798000,shreen bagh,2497
7,G.B Pant Cantonment Padiatric Hospital,34.067159,74.847104,Sonwar,2599
8,SMHS Hospital,34.086124,74.795959,NaN,2588
9,Bone & Joint Hospital,34.048144,74.805243,Ram Bagh,3276


In [56]:
map_your = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, name, address in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_your)  
    
map_your.save('search.html')
map_your